In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pyserini.search.lucene import LuceneSearcher

/storage/home/hcoda1/8/amiyaguchi3/clef/longeval-2025/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pathlib import Path

path = Path("~/scratch/longeval/temp/bm25/index").expanduser()
searcher = LuceneSearcher(path.as_posix())
searcher.set_language("en")
hits = searcher.search("sports")

for i in range(len(hits)):
    print(f"{i + 1:2} {hits[i].docid:4} {hits[i].score:.5f}")

 1 doc3381619 2.37470
 2 doc124910 2.36950
 3 doc92687 2.36950
 4 doc2581092 2.33670
 5 doc462503 2.33510
 6 doc740330 2.30160
 7 doc1926685 2.29720
 8 doc1234486 2.29090
 9 doc1840579 2.28390
10 doc1194131 2.27080


Apr 19, 2025 1:22:53 AM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


In [4]:
from longeval.spark import get_spark
from longeval.collection import ParquetCollection
from pathlib import Path

spark = get_spark(cores=4, memory="10g")

data_root = Path("~/shared/longeval/2025/parquet/").expanduser()
train = ParquetCollection(spark, data_root)
train.queries.printSchema()
train.qrels.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/19 01:22:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/19 01:22:55 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


root
 |-- qid: string (nullable = true)
 |-- query: string (nullable = true)

root
 |-- qid: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- docid: string (nullable = true)
 |-- rel: integer (nullable = true)
 |-- split: string (nullable = true)
 |-- language: string (nullable = true)
 |-- date: string (nullable = true)



In [5]:
from longeval.experiment.bm25.evaluation import (
    prepare_queries,
    run_search,
    score_search,
)

queries = prepare_queries(train)
queries.printSchema()
queries.show(5)

root
 |-- date: string (nullable = true)
 |-- qid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- qrel: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- docid: string (nullable = true)
 |    |    |-- rel: integer (nullable = true)



+-------+---+-----------------+--------------------+
|   date|qid|            query|                qrel|
+-------+---+-----------------+--------------------+
|2022-09|  1|     101boyvideos|[{20007, 0}, {273...|
|2022-08| 10| a vendre chateau|[{5119, 1}, {1678...|
|2022-10| 10| a vendre chateau|[{1592834, 0}, {1...|
|2022-12| 10| a vendre chateau|[{1688208, 0}, {1...|
|2022-07|100|appli pole emploi|[{7753, 0}, {1342...|
+-------+---+-----------------+--------------------+
only showing top 5 rows



In [ ]:
from tqdm import tqdm

index_path = Path("~/scratch/longeval/temp/bm25/index").expanduser().as_posix()

dates = queries.select("date").distinct().orderBy("date").collect()
scores = {}
for date in tqdm(dates):
    search_df = run_search(queries.where(f"date='{date.date}'"), index_path, k=100)
    eval_df = score_search(search_df)
    scores[date] = eval_df

display(scores)

  0%|                                                  | 0/9 [00:00<?, ?it/s]25/04/19 01:23:06 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
25/04/19 01:23:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/04/19 01:23:25 WARN TaskSetManager: Stage 26 contains a task of very large size (4475 KiB). The maximum recommended task size is 1000 KiB.
25/04/19 01:23:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
 11%|████▋                                     | 1/9 [00:27<03:40, 27.58s/it]25/04/19 01:23:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/04/19 01:23:55 WARN TaskSetManager: Stage 40 contains a task of very lar

{Row(date='2022-06'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: string],
 Row(date='2022-07'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: string],
 Row(date='2022-08'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: string],
 Row(date='2022-09'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: string],
 Row(date='2022-10'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: string],
 Row(date='2022-11'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: string],
 Row(date='2022-12'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: string],
 Row(date='2023-01'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: string],
 Row(date='2023-02'): DataFrame[map: double, ndcg: double, ndcg_cut_10: double, ndcg_rel: double, qid: s

In [7]:
list(scores.values())[0].show(5)

Py4JJavaError: An error occurred while calling o232.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 148.0 failed 1 times, most recent failure: Lost task 0.0 in stage 148.0 (TID 152) (atl1-1-02-011-29-2.pace.gatech.edu executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/storage/scratch1/8/amiyaguchi3/longeval/app/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1194, in main
    importlib.invalidate_caches()
  File "/usr/lib/python3.12/importlib/__init__.py", line 70, in invalidate_caches
  File "<frozen importlib._bootstrap_external>", line 1453, in invalidate_caches
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1322, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 1262, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1528, in find_